In [1]:
def conv_size_out(size_in, kern, stride):
    pad = 0
    dilation = 1
    return (size_in + 2*pad - dilation*(kern - 1) - 1) // stride + 1

def avg_size_out(size_in, kern, stride):
    pad = 0
    return (size_in + 2*pad - kern) // stride + 1

def max_size_out(size_in, kern, stride):
    pad = 0
    dilation = 1
    return (size_in + 2*pad - dilation*(kern - 1) - 1) // stride + 1

In [2]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [17]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.model = nn.Sequential(            
#             nn.MaxPool2d(4,2),
            nn.Flatten(),
#             nn.Linear(in_features=225, out_features=10)
        )

    def forward(self, x):
        logits = self.model(x)
        probs = F.softmax(logits, dim=1)
        return logits, probs

In [18]:
x = torch.rand(100,1,32,32)
y = torch.randint(low=0,high=9,size=(100,))

In [19]:
train_dataset = torch.utils.data.TensorDataset(x, y)

train_loader = torch.utils.data.DataLoader(
        dataset = train_dataset,
        batch_size = 32,
        shuffle = True
    )

In [20]:
def profile(model, train_loader):    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)
    
    def train(epochs):
        total_step = len(train_loader)
        print(total_step)

        for epoch in range(epochs):
            for i, (images, labels) in enumerate(train_loader):
                # Forward pass
                outputs = model(images)            
                loss = criterion(outputs, labels)

                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                if (i+1) % 10 == 0:
                    print ('Epoch [{}/{}], Step [{}/{}], Loss: {}' 
                           .format(epoch+1, epochs, i+1, total_step, loss))
    
    EPOCHS = 1
    with torch.autograd.profiler.profile() as prof:
        train(EPOCHS)
        
    return prof.key_averages()

In [21]:
ls = profile(Model().model,train_loader)

ValueError: optimizer got an empty parameter list

In [22]:
print(ls)

------------------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  
Name                                  Self CPU total %  Self CPU total   CPU total %      CPU total        CPU time avg     Number of Calls  
------------------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  
aten::empty                           0.07%            269.152us        0.07%            269.152us        5.277us          51               
aten::random_                         0.01%            50.829us         0.01%            50.829us         25.415us         2                
aten::is_floating_point               0.01%            49.130us         0.01%            49.130us         24.565us         2                
aten::item                            7.39%            27.771ms         16.21%           60.964ms         30.482ms         2                
aten::_local

In [ ]:
for x in ls:
    print(x.key)
    print()

In [10]:
model = Model().model

In [11]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

ValueError: optimizer got an empty parameter list

In [14]:
for i, (images, labels) in enumerate(train_loader):
    # Forward pass
    with torch.autograd.profiler.profile() as prof:
        outputs = model(images)
    break

    loss = criterion(outputs, labels)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    break



In [15]:
print(prof.key_averages())

-----------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  
Name               Self CPU total %  Self CPU total   CPU total %      CPU total        CPU time avg     Number of Calls  
-----------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  
aten::flatten      29.22%           3.572us          100.00%          12.226us         12.226us         1                
aten::reshape      30.56%           3.736us          70.78%           8.654us          8.654us          1                
aten::view         40.23%           4.918us          40.23%           4.918us          4.918us          1                
-----------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  
Self CPU time total: 12.226us



In [ ]:
for k in prof.key_averages():
    print(k.key)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers  
from tensorflow.keras.layers import Dense, Flatten

from numpy.random import RandomState as R

seed = 42

def give(dim, n, channels):
    ds_size = 1024
    out_size = 10
    if dim == 1:
        x = R(seed).random((ds_size, n, channels))
        x = x.reshape(x.shape[0], n, channels)
    else:
        x = R(seed).random((ds_size, n, n, channels))
        x = x.reshape(x.shape[0], n, n, channels)
    
    y = R(seed).randint(0,out_size,ds_size)
    y = tf.keras.utils.to_categorical(y, out_size)
    
    return x,y

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=0.01)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

def profile(model, x, y, batch, epochs):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.batch(batch)
    
    EPOCHS = epochs
    prof_file = 'out_tflow.csv'
    logdir = 'logs'

    with tf.profiler.experimental.Profile(logdir):
        model.fit(dataset, epochs = EPOCHS)
        pass

    return prof_file
    
class Alone:
    def create(self):
        model = Sequential()
        model.add( 
            Dense(units = 32, name = 'DENSE2D')
        )
        model.add( Flatten(name='FLATTEN') )
        model.add( Dense(units = 10, name='FINAL_DENSE') )
        model.compile(loss = loss, optimizer = opt, metrics=['accuracy'])
        self.model = model

Model = Alone()
Model.create()

x,y = give(2, 32, 3)

In [ ]:
prof = profile(Model.model, x, y, 32, 1)

In [11]:
import data_parser as dp

In [12]:
the_dir = ['intermediate_results', 'ptorch']

In [14]:
import os

In [15]:
dct = dp.load(os.path.join(*the_dir, 'dense.torch'))

In [33]:
for x,y in dct.items():
    break

In [36]:
y.sort_values(by='CPU total (us)')

,CPU time avg (us),CPU total (us),Self CPU time total (us),Number of Calls
Name,,,,
is_complex,0.381400,1.907,1.907,5
is_floating_point,0.534000,2.670,2.670,5
torch::autograd::GraphRoot,3.044200,15.221,15.221,5
_local_scalar_dense,1.295880,32.397,32.397,25
detach_,2.116050,42.321,42.321,20
random_,8.900200,44.501,44.501,5
reshape,12.659600,63.298,16.237,5
item,2.763480,69.087,36.690,25
flatten,9.126000,91.260,27.962,10
